In [26]:
# write svm for classification using joint_angles_per_image.csv file
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load the CSV file
df = pd.read_csv("joint_angles_and_distances_per_image.csv")

# Display the first few rows of the dataframe
df.head()




,Image,Left Shoulder-Left Elbow-Left Wrist,Right Shoulder-Right Elbow-Right Wrist,Left Shoulder-Left Hip-Left Knee,Right Shoulder-Right Hip-Right Knee,Left Elbow-Left Shoulder-Left Hip,Right Elbow-Right Shoulder-Right Hip,Left Hip-Left Knee-Left Ankle,Right Hip-Right Knee-Right Ankle,Nose-Left Eye_distance,...,Right Hip_position_x,Left Knee_position_y,Left Knee_position_x,Right Knee_position_y,Right Knee_position_x,Left Ankle_position_y,Left Ankle_position_x,Right Ankle_position_y,Right Ankle_position_x,Label
0,3,176.863623,NaN,104.651653,122.947312,65.335370,NaN,32.463845,49.454794,0.000000,...,483.394226,643.177002,726.250671,239.983398,506.542175,0.000000,0.000000,0.000000,0.000000,push up
1,4,46.826189,69.684937,168.525943,175.557773,1.661186,18.735911,171.895412,169.079502,1161.640015,...,591.639526,553.942078,416.250000,547.771667,402.101929,520.475098,268.811005,535.717529,227.031403,push up
2,5,54.110698,73.605964,169.050747,177.224694,8.860401,14.912523,171.601208,166.037965,1154.134033,...,590.143250,546.691162,410.011993,547.948975,392.688416,514.009399,258.330566,532.663940,218.177002,push up
3,6,73.103679,79.956933,169.036226,178.504071,10.210434,9.026413,170.603113,165.024651,1141.828735,...,589.522949,538.359619,407.604370,546.816284,381.887543,507.196594,257.297791,534.374756,211.694275,push up
4,7,88.872161,82.650630,179.723765,172.554238,24.492551,0.515483,156.767759,151.040331,1130.589844,...,575.672180,556.653381,407.057678,549.610840,377.952026,523.083130,271.386780,519.675415,212.834045,push up


In [27]:
df.shape

(11493, 180)

In [28]:

df.isnull().sum().sort_values(ascending=False)

Left Hip-Left Knee-Left Ankle             2964
Right Hip-Right Knee-Right Ankle          2773
Right Shoulder-Right Hip-Right Knee        844
Left Shoulder-Left Hip-Left Knee           843
Right Shoulder-Right Elbow-Right Wrist     771
                                          ... 
Left Ear-Left Ankle_distance                 0
Left Ear-Right Ankle_distance                0
Right Ear-Left Shoulder_distance             0
Right Ear-Right Shoulder_distance            0
Label                                        0
Length: 180, dtype: int64

In [29]:
# print dropped columns 

dropped_columns = df.columns[df.isnull().sum() > 0]
print(f"Dropped columns: {dropped_columns}")

Dropped columns: Index(['Left Shoulder-Left Elbow-Left Wrist',
       'Right Shoulder-Right Elbow-Right Wrist',
       'Left Shoulder-Left Hip-Left Knee',
       'Right Shoulder-Right Hip-Right Knee',
       'Left Elbow-Left Shoulder-Left Hip',
       'Right Elbow-Right Shoulder-Right Hip', 'Left Hip-Left Knee-Left Ankle',
       'Right Hip-Right Knee-Right Ankle'],
      dtype='object')


In [30]:
# drop nan columns
df = df.dropna(axis=1)


In [31]:
df.shape

(11493, 172)

In [32]:
df.isnull().sum().sum()

0

In [33]:
# encode the labels using LabelEncoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['Label'] = le.fit_transform(df['Label'])

# save the encoder to use it later
import joblib

joblib.dump(le, "models/label_encoder.joblib")


['models/label_encoder.joblib']

In [34]:
# Split the data into features (X) and labels (y)
X = df.drop(columns=["Image", "Label"])  # Features
y = df["Label"]  # Labels

# normalize the data except the label using standard scaler
from sklearn.preprocessing import StandardScaler
standardScaler = StandardScaler()
X = standardScaler.fit_transform(X)
# save the standard scaler for future use
import joblib
joblib.dump(standardScaler, "models/standard_scaler.joblib")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [35]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# train the model with xg boost
from xgboost import XGBClassifier


model = XGBClassifier()
model.fit(X_train, y_train)

# save the model
joblib.dump(model, "models/xgboost_model.joblib")

# Make predictions
y_pred = model.predict(X_test)

# classification report
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# save the model
joblib.dump(model, "models/xgboost_model.joblib")


              precision    recall  f1-score   support

           0       1.00      0.98      0.99       227
           1       1.00      1.00      1.00       357
           2       1.00      1.00      1.00       550
           3       0.99      1.00      0.99       304
           4       1.00      1.00      1.00       861

    accuracy                           1.00      2299
   macro avg       1.00      0.99      1.00      2299
weighted avg       1.00      1.00      1.00      2299

[[223   1   2   1   0]
 [  0 356   0   1   0]
 [  0   0 549   0   1]
 [  0   0   0 303   1]
 [  0   0   0   2 859]]


['models/xgboost_model.joblib']

In [36]:
# train and test svm model
from sklearn.svm import SVC
model = SVC()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# classification report and confusion matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# save the model
joblib.dump(model, "models/svm_model.joblib")



              precision    recall  f1-score   support

           0       0.99      0.90      0.94       227
           1       0.98      0.97      0.98       357
           2       0.97      0.99      0.98       550
           3       0.95      0.98      0.96       304
           4       0.99      0.99      0.99       861

    accuracy                           0.98      2299
   macro avg       0.98      0.97      0.97      2299
weighted avg       0.98      0.98      0.98      2299

[[204   3   9   7   4]
 [  0 348   8   1   0]
 [  1   3 546   0   0]
 [  1   0   0 298   5]
 [  0   0   1   8 852]]


['models/svm_model.joblib']

In [37]:
# train and test random forest model
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# classification report and confusion matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# save the model
joblib.dump(model, "models/random_forest_model.joblib")


              precision    recall  f1-score   support

           0       1.00      0.98      0.99       227
           1       1.00      1.00      1.00       357
           2       0.99      1.00      1.00       550
           3       0.99      1.00      0.99       304
           4       1.00      1.00      1.00       861

    accuracy                           1.00      2299
   macro avg       0.99      0.99      0.99      2299
weighted avg       1.00      1.00      1.00      2299

[[222   1   4   0   0]
 [  0 356   0   1   0]
 [  1   0 549   0   0]
 [  0   0   0 303   1]
 [  0   0   0   2 859]]


['models/random_forest_model.joblib']

In [38]:
# train mlp model
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(250, 150,75,50,25), max_iter=1000)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# classifitaion report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

# save the model
joblib.dump(model, "models/mlp_model.joblib")

Accuracy: 98.74%
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       227
           1       1.00      0.97      0.98       357
           2       0.98      1.00      0.99       550
           3       0.99      0.98      0.98       304
           4       0.99      1.00      0.99       861

    accuracy                           0.99      2299
   macro avg       0.99      0.98      0.98      2299
weighted avg       0.99      0.99      0.99      2299



['models/mlp_model.joblib']

# validation part

### load data

In [39]:
import pandas as pd
# Load the CSV file
df = pd.read_csv("validation_dataset.csv")


In [40]:
df.dropna(axis=1,inplace=True)

In [41]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib

# encode the labels using LabelEncoder
# load the encoder
label_encoder = joblib.load("models/label_encoder.joblib")
df["Label"] = label_encoder.transform(df["Label"])

# Split the data into features (X) and labels (y)
X = df.drop(columns=["Image", "Label"])  # Features
y = df["Label"]  # Labels

# normalize the data except the label using standard scaler
# load model
standardScaler = joblib.load("models/standard_scaler.joblib")
X = standardScaler.fit_transform(X)
# save the standard scaler for future use
import joblib
joblib.dump(standardScaler, "models/standard_scaler.joblib")


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



### test models

In [42]:
# test the xgboost model
model = joblib.load("models/xgboost_model.joblib")
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.32      0.38      0.35       137
           1       0.74      0.69      0.72       186
           2       0.27      0.38      0.31       159
           3       0.09      0.10      0.10       142
           4       0.75      0.63      0.69       543

    accuracy                           0.51      1167
   macro avg       0.43      0.44      0.43      1167
weighted avg       0.55      0.51      0.53      1167

[[ 52   0   9  19  57]
 [  0 129   6  16  35]
 [ 32  40  60  16  11]
 [  1   1 117  14   9]
 [ 78   4  33  85 343]]


In [43]:
# test the svm model
model = joblib.load("models/svm_model.joblib")
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.32      0.41      0.36       137
           1       0.60      0.69      0.64       186
           2       0.33      0.19      0.25       159
           3       0.00      0.00      0.00       142
           4       0.68      0.58      0.63       543

    accuracy                           0.46      1167
   macro avg       0.39      0.37      0.37      1167
weighted avg       0.49      0.46      0.47      1167

[[ 56   0   3  28  50]
 [  0 128  30   9  19]
 [ 17  70  31  37   4]
 [ 29  15  22   0  76]
 [ 73   0   7 147 316]]


In [44]:
# test the random forest model
model = joblib.load("models/random_forest_model.joblib")
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.26      0.36      0.30       137
           1       0.82      0.39      0.53       186
           2       0.24      0.20      0.22       159
           3       0.19      0.32      0.24       142
           4       0.70      0.66      0.68       543

    accuracy                           0.48      1167
   macro avg       0.44      0.39      0.39      1167
weighted avg       0.54      0.48      0.49      1167

[[ 50   0  21  18  48]
 [  8  72  14  23  69]
 [ 65  16  32  33  13]
 [ 30   0  45  46  21]
 [ 39   0  19 127 358]]


In [45]:
# test the mlp model
model = joblib.load("models/mlp_model.joblib")
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))



              precision    recall  f1-score   support

           0       0.71      0.47      0.56       137
           1       0.69      0.77      0.73       186
           2       0.45      0.40      0.42       159
           3       0.39      0.36      0.38       142
           4       0.76      0.83      0.79       543

    accuracy                           0.66      1167
   macro avg       0.60      0.57      0.58      1167
weighted avg       0.66      0.66      0.66      1167

[[ 64   1  33   6  33]
 [  0 144  22   7  13]
 [  3  62  64   1  29]
 [  7   0  16  51  68]
 [ 16   2   8  65 452]]
